In [41]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ast

from os import listdir
from os.path import isfile, join


In [42]:
# EXTRACT SENTIMENT AGGREGATED BY TIMESPAN

ticker = 'MSFT'
news =  pd.read_csv('/home/simone/Desktop/NLFF/GoogleNL/SentimentSingleNews/'+str(ticker)+'_googleSentVector.csv')
news.drop_duplicates(subset=['DATE'], inplace=True)
news = news.sort_values(by=['DATE'])
news = news.reset_index(drop=True)
news = news.drop(['Unnamed: 0'], axis=1)
news['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S ') for row in news['DATE']]
#news['entSentVec'] = [ast.literal_eval(row) for row in news['entSentVec']]

print(list(news.columns))


['DATE', 'GoogleSentiment', 'microsoft_sa', 'stock_sa', 'compani_sa', 'share_sa', 'appl_sa', 'nasdaq_sa', 'amazon_sa', 'googl_sa', 'earn_sa', 'investor_sa', 'investorplace stock market_sa', 'market_sa', 'facebook_sa', 'posit_sa', 'world_sa', 'game_sa', 'revenu_sa', 'busi_sa', 'microsoft corpor_sa', 'amp trading tip_sa', 'cloud_sa', 'u.s._sa', 'technolog_sa', 'growth_sa', 'result_sa', 'thing_sa', 'amp_sa', 'analyst_sa', 'report_sa', 'ibm_sa', 'microsoft_se', 'stock_se', 'compani_se', 'share_se', 'appl_se', 'nasdaq_se', 'amazon_se', 'googl_se', 'earn_se', 'investor_se', 'investorplace stock market_se', 'market_se', 'facebook_se', 'posit_se', 'world_se', 'game_se', 'revenu_se', 'busi_se', 'microsoft corpor_se', 'amp trading tip_se', 'cloud_se', 'u.s._se', 'technolog_se', 'growth_se', 'result_se', 'thing_se', 'amp_se', 'analyst_se', 'report_se', 'ibm_se', 'microsoft_ma', 'stock_ma', 'compani_ma', 'share_ma', 'appl_ma', 'nasdaq_ma', 'amazon_ma', 'googl_ma', 'earn_ma', 'investor_ma', 'invest

In [43]:


timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/indexes/indexes"+str(ticker)+".csv")
timeSpan = timeSpan['date'].tolist()
# This dataset is already GMT+0
timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]

features_names = news.columns.tolist()
features_names.remove('DATE')
features_names.remove('GoogleSentiment')
concepts = list(set([f[:-3] for f in features_names]))
concepts.remove('tot')
print(concepts)
sentiment = pd.DataFrame(columns=['initTime']+features_names)
#print(list(sentiment.columns))

#Prendiamo intervallo massimo dove ci sono entrambi i dataset:
#massimo della data iniziale tra i due
#minimo della data finale tra i due
initDate = max(timeSpan[0], news['DATE'][0])
finalDate = min(timeSpan[len(timeSpan)-1], news['DATE'][len(news)-1])



i = 0
j = 0
# ALLINEAMENTO INIZIO
while(timeSpan[j] < initDate):
    j+=1
init = j
while(news['DATE'][i] <= initDate - timedelta(hours=1)):
    i+=1

print(timeSpan[j])
print(news['DATE'][i])
    
print("Start: "+str(initDate) +"("+str(timeSpan[j])+") i: "+str(i)+" j: "+str(j))
print("End: "+str(finalDate))

#ALLINEAMENTO FINE DENTRO I WHILE
while( news['DATE'][i] < finalDate and timeSpan[j] < finalDate ):
    initTime = timeSpan[j]
    normal_sum = {feature_name: 0 for feature_name in features_names}    
    num_tot_news = 0
    while(i<len(news)-1 and timeSpan[j] > news['DATE'][i]):
        if(i%1000 == 0):
            print(i)
        if not (timeSpan[j] > news['DATE'][i] and timeSpan[j-1] <= news['DATE'][i]):
            print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
            assert False
#         a = news['entSentVec'][i]
#         print(type(a))
#         print(len(a))
#         a = np.asarray(news['entSentVec'][i])
#         print(type(a))
#         print(a.shape)
            
        normal_sum = {f : normal_sum[f]+news[f][i] for f in features_names}        
        num_tot_news +=1
        i+=1
    j+=1
    #sentiment.loc[j] = {'initTime':initTime, 'num_news':num_tot_news}
    sentiment.loc[j,'initTime'] = initTime
    sentiment.loc[j,'tot_co'] = num_tot_news
      
    for c in concepts:
        normal_sum[c+'_sa'] /= normal_sum[c+'_co'] if normal_sum[c+'_co'] > 0 else 1
        normal_sum[c+'_se'] /= normal_sum[c+'_co'] if normal_sum[c+'_co'] > 0 else 1
        normal_sum[c+'_ma'] /= normal_sum[c+'_co'] if normal_sum[c+'_co'] > 0 else 1
    for f in normal_sum.keys():
        sentiment.loc[j,f] = normal_sum[f]
    


end = j
if(len(sentiment) != end - init):
    print(len(sentiment))
    print(end - init + 1)
    assert False


#sentiment.to_csv('Aggregated_1h_dataset/'+str(ticker)+'_googleSentPerTimestep.csv')

['stock', 'investor', 'analyst', 'amazon', 'nasdaq', 'market', 'amp', 'amp trading tip', 'investorplace stock market', 'posit', 'world', 'result', 'googl', 'microsoft corpor', 'ibm', 'earn', 'busi', 'thing', 'report', 'cloud', 'revenu', 'game', 'technolog', 'growth', 'u.s.', 'appl', 'facebook', 'microsoft', 'share', 'compani']
2017-05-22 14:00:00
2017-05-22 00:46:00
Start: 2017-05-22 00:46:00(2017-05-22 14:00:00) i: 0 j: 4051
End: 2018-06-20 23:29:41
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [44]:
sentiment = sentiment.sort_values(by=['initTime'])
sentiment = sentiment.reset_index(drop=True)
sentiment.head()

,initTime,microsoft_sa,stock_sa,compani_sa,share_sa,appl_sa,nasdaq_sa,amazon_sa,googl_sa,earn_sa,...,growth_co,result_co,thing_co,amp_co,analyst_co,report_co,ibm_co,tot_se,tot_ma,tot_co
0,2017-05-22 14:00:00,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,1.2,7.0
1,2017-05-22 15:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
2,2017-05-22 16:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.2,2.0
3,2017-05-22 17:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
4,2017-05-22 18:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0


In [45]:
sentiment.to_csv('Aggregated_1h_dataset/'+str(ticker)+'_googleSentPerTimestep.csv')